In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -q datasets transformers rouge-score nltk sentencepiece

     |████████████████████████████████| 362 kB 5.2 MB/s 
     |████████████████████████████████| 4.4 MB 64.2 MB/s 
     |████████████████████████████████| 1.2 MB 46.7 MB/s 
     |████████████████████████████████| 1.1 MB 41.3 MB/s 
     |████████████████████████████████| 86 kB 5.2 MB/s 
     |████████████████████████████████| 212 kB 45.7 MB/s 
     |████████████████████████████████| 140 kB 54.4 MB/s 
     |████████████████████████████████| 596 kB 44.5 MB/s 
     |████████████████████████████████| 127 kB 57.3 MB/s 
     |████████████████████████████████| 6.6 MB 45.9 MB/s 
     |████████████████████████████████| 271 kB 43.8 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |████████████████████████████████| 144 kB 26.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which i

In [4]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer, DataCollatorForSeq2Seq, EarlyStoppingCallback
from datasets import load_dataset, load_metric, Dataset
import pandas as pd
import numpy as np
import re
import nltk
import torch
import os

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
model_name = 'ainize/kobart-news'
metric_name = 'rouge'

In [7]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
metric = load_metric(metric_name)

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/302 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [8]:
prefix = ""
# prefix = "summarize: "

max_input_length = 1024
max_target_length = 512

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["context"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    # with tokenizer.as_target_tokenizer():
    #     labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    # model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [9]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [13]:
!gdown -q 1BPI0rGQ1LseACn8kPVmaKreiU3VBgoxh

In [15]:
test_path = '/content/test_df.csv'

In [16]:
test_df = pd.read_csv(test_path)

In [17]:
dataset = Dataset.from_pandas(test_df)
dataset = dataset.map(preprocess_function, batched=True)

Parameter 'function'=<function preprocess_function at 0x7fead1de25f0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

In [23]:
per_device_eval_batch_size = 1
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)

In [24]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./log",

    per_device_eval_batch_size=per_device_eval_batch_size,

    predict_with_generate=True,
    generation_max_length=512,
    generation_num_beams=5,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [25]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=data_collator,
    # compute_metrics=compute_metrics,
)

In [26]:
preds = trainer.predict(dataset)

The following columns in the test set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: context, Unnamed: 0, total, title, token_type_ids, agenda, id, region. If context, Unnamed: 0, total, title, token_type_ids, agenda, id, region are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 506
  Batch size = 8


KeyboardInterrupt: ignored

In [ ]:
for data, pred in zip(dataset, preds.predictions):
    context = tokenizer.decode(data['input_ids'], skip_special_tokens=True)

    pred = tokenizer.decode(pred[2:], skip_special_tokens=True)
    print(f'입력: {context}')

    print(f'예측: {pred}\n')